In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
import matplotlib.pyplot as plt

In [ ]:
def load_and_preprocess_data(file_path, selected_columns):
    data = pd.read_csv(file_path)
    data = data[selected_columns]
    data = data.dropna()
    pressure_mean = data['Pressure'].mean()  # Calculate the mean of the Pressure column
    print("Mean Pressure ", pressure_mean)
    print(data[data['Pressure'] == 0].count())
    data.loc[data['Pressure'] == 0, 'Pressure'] = pressure_mean  # Replace all the rows with the mean value if the pressure is recorded as zero
    print(data[data['Pressure'] == 0].count())
    return data

In [ ]:
def split_features_target(data, target_column):
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    return X, y

In [ ]:
def split_train_test(X, y, test_size=0.3, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

In [ ]:
def scale_data(X_train, X_test, y_train, y_test):
    feature_scaler = StandardScaler()
    X_train_scaled = feature_scaler.fit_transform(X_train)
    X_test_scaled = feature_scaler.transform(X_test)

    target_scaler = MinMaxScaler()
    y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, feature_scaler, target_scaler

In [ ]:
def build_lstm_model(input_shape, hidden_units=(64, 128, 128, 64, 32, 8), learning_rate_init=0.004):
    model = Sequential()
    model.add(LSTM(hidden_units[0], input_shape=input_shape, return_sequences=True))
    for units in hidden_units[1:-1]:
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(hidden_units[-1]))  # Last LSTM layer without return_sequences
    model.add(Dense(1))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_init)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [ ]:
class MetricsHistory(Callback):
    def __init__(self, X_test, y_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.mse_history = []
        self.r2_history = []
        self.mae_history = []
        self.rmse_history = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_test).flatten()
        mse = mean_squared_error(self.y_test, y_pred)
        mae = mean_absolute_error(self.y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(self.y_test, y_pred)
        self.mse_history.append(mse)
        self.mae_history.append(mae)
        self.rmse_history.append(rmse)
        self.r2_history.append(r2)

In [ ]:
def train_and_record_metrics(X_train_scaled, y_train, X_test_scaled, y_test, input_shape, hidden_units, learning_rate):
    model = build_lstm_model(input_shape=input_shape, hidden_units=hidden_units, learning_rate_init=learning_rate)
    metrics_history = MetricsHistory(X_test_scaled, y_test)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)
    model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
              epochs=300, batch_size=256, callbacks=[metrics_history, lr_scheduler], verbose=0)

    y_pred = model.predict(X_test_scaled).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return metrics_history, mse, mae, r2

In [ ]:
def plot_metrics(metrics_histories, labels):
    plt.figure(figsize=(20, 12))

    plt.subplot(2, 2, 1)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.mse_history, label=f'{label}', linewidth=2)
    plt.xlabel('Epoch', fontsize=18)
    plt.ylabel('MSE', fontsize=18)
    plt.title('MSE vs Epochs', fontsize=20)
    plt.legend(fontsize=20)

    plt.subplot(2, 2, 2)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.mae_history, label=f'{label}', linewidth=2)
    plt.xlabel('Epoch', fontsize=18)
    plt.ylabel('MAE', fontsize=18)
    plt.title('MAE vs Epochs', fontsize=20)
    plt.legend(fontsize=20)

    plt.subplot(2, 2, 3)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.rmse_history, label=f'{label}', linewidth=2)
    plt.xlabel('Epoch', fontsize=18)
    plt.ylabel('RMSE', fontsize=18)
    plt.title('RMSE vs Epochs', fontsize=20)
    plt.legend(fontsize=20)

    plt.subplot(2, 2, 4)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.r2_history, label=f'{label}', linewidth=2)
    plt.xlabel('Epoch', fontsize=18)
    plt.ylabel('R² Score', fontsize=18)
    plt.title('R² vs Epochs', fontsize=20)
    plt.legend(fontsize=20)

    plt.tight_layout()
    plt.show()

In [ ]:
def ablation_layer_study(selected_columns, target_column='Level'):
    merged_df = load_and_preprocess_data('./Datasets/2021-Kippure.csv', selected_columns)

    X, y = split_features_target(merged_df, target_column)
    X_train, X_test, y_train, y_test = split_train_test(X, y)
    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, feature_scaler, target_scaler = scale_data(X_train, X_test, y_train, y_test) # Calling the function to scale the data. We have used StandardScaler and MinmaxScaler to scale the data. 

#  Defined the model parameters below
    learning_rate = 0.001
    hidden_layer_configurations = [
        (64, 128, 128, 64),
        (64, 128, 64),
        (64, 64),
    ]
    metrics_histories = []
    labels = []
    input_shape = (1, X_train_scaled.shape[1])  # Adjust input shape for LSTM

    # Initialize an empty DataFrame to store the metrics
    metrics_df = pd.DataFrame(columns=['Architecture', 'MSE', 'MAE', 'R²'])

    for hidden_layers in hidden_layer_configurations:
        metrics_history, mse, mae, r2 = train_and_record_metrics(
            X_train_scaled.reshape(-1, 1, X_train_scaled.shape[1]),  # Reshaping the input to the model.
            y_train_scaled, 
            X_test_scaled.reshape(-1, 1, X_test_scaled.shape[1]), 
            y_test_scaled, 
            input_shape, hidden_layers, learning_rate
        )
        metrics_histories.append(metrics_history)
        labels.append(f"Layers {hidden_layers}")
        
        # Append the final metrics to the DataFrame
        metrics_df = pd.concat([metrics_df, pd.DataFrame([{
            'Architecture': hidden_layers,
            'MSE': mse,
            'MAE': mae,
            'R²': r2
        }])], ignore_index=True)
    
    # Print the DataFrame with the metrics
    print(metrics_df)

    plot_metrics(metrics_histories, labels)

In [ ]:
if __name__ == "__main__":
    selected_columns = ['Windspeed', 'Humidity', 'Temperature', 'Dewpoint', 'Pressure', 'Reading', 'Wind direction', 'Level']
    ablation_layer_study(selected_columns)